<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Sacasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/My Drive/

/content/drive/My Drive


In [4]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Loading data from json file
data = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines = True)
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [7]:
# getting features and labels
features = data['headline']
labels = data['is_sarcastic']

In [8]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [9]:
# Stemming our data
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer()
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [11]:
tf_idf_x = tfIdfVec.transform(features)

In [12]:
# getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.todense(), labels, test_size = .1, random_state = 0)

In [13]:
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print("Train accuracy: ",lsvc.score(features_train, labels_train))
print("Test accuracy: ",lsvc.score(features_test, labels_test))

Train accuracy:  0.9538377916682843
Test accuracy:  0.8371767994409504


In [14]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_estimators=500,random_state=42)
# rfc.fit(features_train,labels_train)
# preds = rfc.predict(features_test)
# print('accuracy: ', accuracy_score(labels_test,preds))

In [15]:
# Loading data from json file
data1 = pd.read_json("dataset.json", lines = True, encoding='cp1252' )
data1.head()

,is_sarcastic,headline,article_link
0,0,The Police Officers Who Killed Breonna Taylor ...,https://www.huffingtonpost.co.uk/entry/police-...
1,1,Scientists Discover Mysterious Radio Transmiss...,https://www.theonion.com/scientists-discover-m...
2,0,Historic Anti-Lynching Legislation Blocked In ...,https://www.huffingtonpost.co.uk/entry/histori...
3,1,Democratic Leaders Announce That They’ve Learn...,https://politics.theonion.com/democratic-leade...
4,0,Will A Coronavirus Vaccine Really Be 'Availabl...,https://www.huffingtonpost.co.uk/entry/coronav...


In [16]:
features_val = data1['headline']
labels_val = data1['is_sarcastic']

In [17]:
# Stemming our data
ps1 = PorterStemmer()
features_val = features_val.apply(lambda x: x.split())
features_val = features_val.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

In [18]:
tv1 = tfIdfVec
#features_val = list(features_val)
features_val = tv1.transform(features_val).toarray()

In [19]:
# getting the score of train and test data
print("Val accuracy: ",lsvc.score(features_val, labels_val))

Val accuracy:  0.7770853307766059
